In [ ]:
%matplotlib inline
%run ../setup/nb_setup

# Orbits 3: Orbits in Triaxial Potentials

Author(s): Adrian Price-Whelan


## Learning goals

In this tutorial, we will introduce triaxial potential models, and explore the additional complexity that this brings to the landscape of orbits, as compared to orbits in axisymmetric potential models.


## Introduction

In the last tutorial, we saw that reducing the symmetry of a potential from spherical to axisymmetric reduced the number if isolating integrals of motion from four to three. Will removing another potential symmetry — going from axisymmetric to triaxial — further reduce the number of isolating integrals? Luckily no: It turns out that regular orbits in triaxial potentials still have three isolating integrals. However, the orbit structure of triaxial potentials is more complex: A new type of orbit family can exist in triaxial potentials known as "box orbits," and tube orbits (what we are familiar with from axisymmetric models) can only exist around either the short or long axis of the potential. Triaxial potentials also generally have larger regions of chaotic orbits, making chaos more relevant in triaxial mass models.

In this tutorial, we will introduce some triaxial gravitational potential models, demonstrate the different types of orbits that can exist in triaxial models, and compare chaotic and regular orbits.


## Terminology and Notation

- (See Orbits tutorials 1 and 2)

### Notebook Setup and Package Imports

In [ ]:
from astropy.constants import G
import astropy.units as u

from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

## Triaxial Potential Models

* ** Triaxial models
    * Angular momentum components not conserved, but idea of frequencies still relevant
    * Potential resonances, analogy to closed orbits in spherical (e.g., Kepler)
    * Existence of fundamental frequencies implies existence of integrals of motion

To recap the last tutorial, we found that we could define axisymmetric potential–density pairs by replacing the spherical radius with a elliptical radius that scales the $z$ axis differently. Axisymmetric models are useful for defining potentials for flattened mass distributions, such as from galaxy disks. Triaxial potential models are models in which the isopotential or isodensity contours coincide with ellipsoidal surfaces. These models are useful for representing the outskirts of dark matter halos (which have been found to be triaxial, to first order) and for representing the stellar distributions in elliptical galaxies. 

To create a triaxial potential, we can  instead replace the spherical radius with an ellipsoidal radius defined as

$$
r^2_{\rm ell} = \frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2}
$$

where often by convention, $a$ is set to $a=1$ so that $b$ is the $y/x$ *axis ratio* and $c$ is the $z/x$ axis ratio. As with the axisymmetric models, we can either replace the radius in the expression of a spherical potential, or replace the radius in the expression of the density function, but these have different effects. Ideally, we would want to replace the radius in the form of a spherical density with the ellipsoidal radius, because then the density would be guaranteed to be positive and physical at all positions. However, there are very few triaxial solutions to Poisson's equation known, so most triaxial density distributions do not have analytic expressions for the potential (or gradients, which we need to compute orbits). It is therefore common to instead replace that radius in a potential expression with an ellipsoidal radius, but this has the same caveat as we saw with axisymmetric models that for some choices of $b, c$ the density can become negative. 


### Example: The Triaxial Log Potential

One example of such a potential model is the triaxial extension of the flattened log potential model discussed in the previous tutorial. The form of the potential is given by

$$
\Phi_L(\boldsymbol{x}) = \frac{1}{2}\,v_0^2 \, \ln\left(\frac{x^2}{q_1^2} + \frac{y^2}{q_2^2} + 
    \frac{z^2}{q_3^2} + r_h^2 \right)
$$

where the axis ratios are given by the parameters $(q_1, q_2, q_3)$, and by convention we typically set $q_1=1$.

## Integrals of Motion and Orbit Classes in Triaxial Models

Regular orbits in triaxial potential models still have three integrals of motion, however, like in the axisymmetric case, these are again not easily associated with classical integrals or analytic expressions. Regular orbits do have actions, but the interpretation of the actions often depends on the type of orbit being studied. To understand this a bit more, we need to look at the different types of orbit families that exist in triaxial potential models. Recall that in spherical models, all orbits are planar, and generic orbits form rosette patterns because of radial and azimuthal oscillations that often have an irrational frequency ratio. In axisymmetric models, orbits conserve the $z$ component of their angular momentum and so preserve their circulation around the symmetry axis of the potential, forming vertically-thickened analogs to the planar rosette patterns called "tube" orbits. In triaxial models, there are four general classes of orbits: short-axis tube orbits, "inner" long-axis tube orbits, "outer" long-axis tube orbits, and box orbits. 


### Tube Orbits

Tube orbits in triaxial models can only exist around the long and short axes of the potential, but around these axes the orbits look a lot like their axisymmetric counterparts. In the exercise below, we will compute a few tube orbits and take a closer look at their properties.


#### Exercise: Long- and short-axis tube orbits

First define a Gala `LogarithmicPotential` object with:
* $v_c=230~\textrm{km}~\textrm{s}^{-1}$
* $r_h=15~\textrm{kpc}$
* $q_1 = 1$
* $q_2 = 0.9$
* $q_3 = 0.8$

In [ ]:
# Define the potential object here
# triaxial_log =

triaxial_log = gp.LogarithmicPotential(
    v_c=230 * u.km / u.s, r_h=15 * u.kpc, q1=1.0, q2=0.9, q3=0.8, units=galactic
)

Now set up three initial conditions: 

* At $\boldsymbol{x} = (10, 1, 3)~\textrm{kpc}$, with $v_y = v_c$
* At $\boldsymbol{x} = (3, 10, 1)~\textrm{kpc}$, with $v_z = v_c$
* At $\boldsymbol{x} = (1, 20, 1)~\textrm{kpc}$, with $v_x=v_c/2$, $v_z = v_c/2$

(other velocity components set to 0)

In [ ]:
# Define the initial conditions here
# tube_w0s = ...

tube_w0s = gd.PhaseSpacePosition(
    pos=([[10, 1, 3.0], [3, 10, 1.0], [1, 20, 1.0]] * u.kpc).T,
    vel=([[0, 230, 0.0], [0, 0, 230.0], [115, 0, 115]] * u.km / u.s).T,
)

Integrate these three orbits for a total integration time of 100 Gyr with a timestep of 2 Myr:

In [ ]:
# Integrate the orbits here:
# tube_orbits = ...

tube_orbits = triaxial_log.integrate_orbit(
    tube_w0s, dt=2 * u.Myr, t1=0, t2=100 * u.Gyr, Integrator=gi.DOPRI853Integrator
)

Plot the three orbits in all 2D projections of the 3D positions (x-y, x-z, y-z):

In [ ]:
for i in range(tube_orbits.norbits):
    _ = tube_orbits[:, i].plot()

Compute and plot the angular momentum components for the three orbits as a function of time:

In [ ]:
# tube_orbits_L = ...

tube_orbits_L = tube_orbits.angular_momentum()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
for i, (ax, lbl) in enumerate(zip(axes, ["$L_x$", "$L_y$", "$L_z$"])):
    ax.plot(tube_orbits.t, tube_orbits_L[i])
    ax.set_ylabel(lbl)
    ax.set_xlabel(f"time [{tube_orbits.t.unit:latex_inline}]")

In [ ]:
azims = np.linspace(0, 180, 128)
half_elevs = np.linspace(-80, 80, len(azims) // 2)
elevs = np.concatenate((half_elevs, half_elevs[::-1]))
fig, anim = animate_3d(tube_orbits[:, 2], azims=azims, elevs=elevs, total_time=8)
plt.close(fig)
HTML(anim.to_html5_video())

In [ ]:
for i in range(tube_orbits.norbits):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5), subplot_kw=dict(projection="3d"))
    _ = tube_orbits[:, i].plot_3d(ax=ax)
    ax.azim = 30
    lim = (-20, 20)
    ax.set(xlim=lim, ylim=lim, zlim=lim)

In [ ]:
triaxial_log.to_sympy()[0]

### Exercise: something about trying to compute a tube orbit about intermediate axis

In [ ]:
w0_x = gd.PhaseSpacePosition(pos=[0, 10.0, 0] * u.kpc, vel=[5, 5.0, 200.0] * u.km / u.s)

w0_y = gd.PhaseSpacePosition(pos=[10.0, 0, 0] * u.kpc, vel=[0, 5.0, 200.0] * u.km / u.s)

w0_z = gd.PhaseSpacePosition(
    pos=[10.0, 0, 0] * u.kpc, vel=[0, 200.0, 15.0] * u.km / u.s
)

w0_box = gd.PhaseSpacePosition(pos=[10, 2, 5] * u.kpc, vel=[0, 0, 0.0] * u.km / u.s)

w0s = gd.combine((w0_x, w0_y, w0_z, w0_box))

In [ ]:
orbits = triaxial_log.integrate_orbit(
    w0s, dt=0.5 * u.Myr, t1=0, t2=8 * u.Gyr, Integrator=gi.DOPRI853Integrator
)

In [ ]:
_ = orbits.plot_3d()

## Irregular and Chaotic Orbits
    
* ** Chaotic orbits
    * Lyapunov exponent
    * Estimation of fundamental frequencies (conceptual / FFT)
    * Frequency diffusion